In [1009]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib as plt
from torch.utils.data import DataLoader
from sklearn.model_selection import StratifiedKFold
import numpy as np

In [1010]:

class BinaryClassifier(nn.Module):
    def __init__(self, input_dim, hidden_layers):
        super(BinaryClassifier, self).__init__()
        layers = []
        #add fully connected layers with the input dim dividing by two each time the input shape
        output_dim = max(3*input_dim//(4), 1)
        for i in range(hidden_layers):
            i += 1
            layers.append(nn.Linear(input_dim,output_dim))
            layers.append(nn.ReLU())
            input_dim = output_dim
            output_dim = max(int(input_dim//(0.7)), 1)
        #add final layer with sigmiod output and final linear layer
        layers.append(nn.Linear(input_dim,1))
        #layers.append(nn.Sigmoid())

        #create a Seqeuntial implementation, no additional layers but provides one line to run through all layers
        self.network = nn.Sequential(*layers)
    
    def forward(self, x):
        #apply the neural net
        return self.network(x)

In [1011]:
#input shape is not correct
input_shape = 30
hidden_layers = 6
model_1 = BinaryClassifier(input_shape, hidden_layers)

In [1012]:
from sklearn.model_selection import train_test_split
import pandas as pd
from torch.utils.data import DataLoader, TensorDataset

In [1013]:

def dataset_to_numpy(dataset: TensorDataset):
    X, y = dataset.tensors
    # Ensure on CPU and detached from computation graph
    X_np = X.detach().numpy()
    y_np = y.detach().numpy()
    return X_np, y_np


In [1014]:
def load_data():
    # Read data
    df = pd.read_csv('br_cancer_normalized.csv', encoding='latin-1')

    # Separate features and labels
    X = df.drop(columns=['Diagnosis']).to_numpy()
    y = df['Diagnosis'].to_numpy()
    return X, y

In [1015]:
#import whatever data loader function is needed
#should output train dataloader and test dataloader  as the full train and test datasets
X_np, y_np = load_data()
print(X_np.shape, y_np.shape)

(569, 30) (569,)


In [1016]:
# Count positives and negatives
num_pos = (y_np == 1).sum().item()
num_neg = (y_np == 0).sum().item()

# Compute base pos_weight
base_pos_weight = num_neg / num_pos

# Increase penalty for missing a 1 by 20%
pos_weight_value = torch.tensor(base_pos_weight*2)

In [1017]:
optimizer = optim.Adam(model_1.parameters(), lr=1e-3)
loss_fn = nn.BCEWithLogitsLoss(pos_weight=pos_weight_value * 1.2)

In [1018]:

def visualize_prediction_distribution(model, dataloader, threshold=0.5):
    """
    Visualize the percentage of predicted 0s and 1s from a binary classifier.
    
    Args:
        model: Trained PyTorch model
        dataloader: DataLoader containing your dataset
        threshold: Decision threshold for classification (default=0.5)
        device: "cpu" or "cuda"
    """
    model.eval()
    preds = []
    
    with torch.no_grad():
        for batch_X, _ in dataloader:
            outputs = model(batch_X).squeeze()
            
            # If model ends with Sigmoid
            if outputs.dim() == 0:
                outputs = outputs.unsqueeze(0)
            
            predicted = (outputs >= threshold).int().cpu().numpy()
            preds.extend(predicted)
    
    preds = np.array(preds)
    total = len(preds)
    percent_0 = (preds == 0).sum() / total * 100
    percent_1 = (preds == 1).sum() / total * 100
    print(percent_0, "this is percent 0")
    # # Bar plot
    # plt.bar(["Predicted 0", "Predicted 1"], [percent_0, percent_1], color=["skyblue", "salmon"])
    # plt.ylabel("Percentage (%)")
    # plt.title("Prediction Distribution")
    # plt.show()
    
    print(f"Predicted 0: {percent_0:.2f}%")
    print(f"Predicted 1: {percent_1:.2f}%")


In [1019]:
from statistics import mean, stdev
from sklearn import preprocessing
from sklearn.model_selection import StratifiedKFold
from sklearn import linear_model
from sklearn import datasets

In [1020]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader, Subset
from sklearn.model_selection import StratifiedKFold
import copy

def test(model, data, criterion):
    model.eval()
    test_loss = 0.0
    correct = 0
    total = 0
    
    with torch.no_grad():
        for batch_X, batch_y in data:
            batch_y = batch_y.unsqueeze(1)
            logits = model(batch_X)
            loss = criterion(logits, batch_y)
            test_loss += loss.item() * batch_X.size(0)

            preds = (torch.sigmoid(logits) >= 0.5).float()
            correct += (preds == batch_y).sum().item()
            total += batch_y.size(0)
    
    avg_loss = test_loss / total
    accuracy = correct / total
    print(f"Test Loss: {avg_loss:.4f} | Test Acc: {accuracy:.4f}")
    return avg_loss, accuracy
    

def train_with_stratified_kfold(X, y, model_fn, input_shape, hidden_layers, batch_size=32, lr=1e-3, k_folds=5, num_epochs=100):
    # Wrap into dataset
    dataset = TensorDataset(torch.tensor(X, dtype=torch.float32),
                            torch.tensor(y, dtype=torch.float32))
    
    skf = StratifiedKFold(n_splits=k_folds, shuffle=True)
    all_fold_results = []
    models = []
    for fold, (train_idx, val_idx) in enumerate(skf.split(X, y)):
        print(f"\n--- Fold {fold+1} ---")

        train_subset = Subset(dataset, train_idx)
        val_subset = Subset(dataset, val_idx)

        train_loader = DataLoader(train_subset, batch_size=batch_size, shuffle=True)
        val_loader = DataLoader(val_subset, batch_size=batch_size, shuffle=False)
     # fresh model for each fold
        cur_model = BinaryClassifier(30,5)
        criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight_value * 1.05)
        optimizer = optim.Adam(cur_model.parameters(), lr=lr)

        for epoch in range(num_epochs):
            cur_model.train()
            epoch_loss = 0.0
            correct = 0
            total = 0

            for batch_X, batch_y in train_loader:
                optimizer.zero_grad()
                outputs = cur_model(batch_X).squeeze()
                loss = criterion(outputs, batch_y)
                loss.backward()
                optimizer.step()

                epoch_loss += loss.item() * batch_X.size(0)
                preds = (torch.sigmoid(outputs) >= 0.5).float()
                correct += (preds == batch_y).sum().item()
                total += batch_y.size(0)

            train_acc = correct / total
            train_loss = epoch_loss / total
            print(f"Epoch {epoch+1}: Train Loss {train_loss:.4f} | Train Acc {train_acc:.4f}")

        avg_loss, accuracy = test(cur_model, val_loader, criterion)
        all_fold_results.append((avg_loss, accuracy))
        models.append(cur_model)
        visualize_prediction_distribution(cur_model, val_loader)

    
    return all_fold_results , models


In [1021]:
fold_results, models = train_with_stratified_kfold(X_np,y_np,model_1,30,5)


--- Fold 1 ---
Epoch 1: Train Loss 1.3572 | Train Acc 0.5099
Epoch 2: Train Loss 1.2737 | Train Acc 0.3736
Epoch 3: Train Loss 1.1690 | Train Acc 0.3736
Epoch 4: Train Loss 1.0877 | Train Acc 0.3736
Epoch 5: Train Loss 0.9454 | Train Acc 0.3736
Epoch 6: Train Loss 0.7388 | Train Acc 0.4242
Epoch 7: Train Loss 0.5802 | Train Acc 0.8418
Epoch 8: Train Loss 0.4566 | Train Acc 0.8945
Epoch 9: Train Loss 0.3287 | Train Acc 0.9209
Epoch 10: Train Loss 0.2435 | Train Acc 0.9341
Epoch 11: Train Loss 0.2680 | Train Acc 0.9451
Epoch 12: Train Loss 0.1915 | Train Acc 0.9363
Epoch 13: Train Loss 0.1903 | Train Acc 0.9516
Epoch 14: Train Loss 0.1961 | Train Acc 0.9538
Epoch 15: Train Loss 0.1510 | Train Acc 0.9626
Epoch 16: Train Loss 0.1456 | Train Acc 0.9560
Epoch 17: Train Loss 0.1684 | Train Acc 0.9648
Epoch 18: Train Loss 0.1573 | Train Acc 0.9648
Epoch 19: Train Loss 0.1325 | Train Acc 0.9736
Epoch 20: Train Loss 0.1618 | Train Acc 0.9692
Epoch 21: Train Loss 0.1532 | Train Acc 0.9582
Epoch 

In [1022]:
print(fold_results)

[(0.20836834148544686, 0.9736842105263158), (0.13208506143602886, 0.9824561403508771), (0.20606886708226643, 0.9649122807017544), (0.6065831927556071, 0.9736842105263158), (0.29395549117991354, 0.9911504424778761)]


In [1023]:
best_model = models[4]

In [1024]:
print(best_model)

BinaryClassifier(
  (network): Sequential(
    (0): Linear(in_features=30, out_features=22, bias=True)
    (1): ReLU()
    (2): Linear(in_features=22, out_features=31, bias=True)
    (3): ReLU()
    (4): Linear(in_features=31, out_features=44, bias=True)
    (5): ReLU()
    (6): Linear(in_features=44, out_features=62, bias=True)
    (7): ReLU()
    (8): Linear(in_features=62, out_features=88, bias=True)
    (9): ReLU()
    (10): Linear(in_features=88, out_features=1, bias=True)
  )
)
